In [500]:
import pandas as pd
import numpy as np
import re

### №1 Завантаження даних і фільтрування за ключовими словами

In [501]:
"""
ЗАВАНТАЖЕННЯ ФАЙЛУ З BIPRO.PROZORRO ІЗ ЗАВЕРШЕНИМИ ЗАКУПІВЛЯМИ МІСЬКИХ РАД.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк.
Дата - від 1 серпня 2016 року.
"""


df_raw = pd.read_csv('24RegionsFull.csv', sep=';', skiprows = [1], low_memory = False)
print(len(df_raw))
print(df_raw['Главный орган'].unique().tolist())
print(df_raw.columns)

592175
['Львівська міська рада', 'Дніпровська міська рада', 'КМДА', 'Одеська міська рада', 'Сєвєродонецька міська рада', 'Запорізька міська рада', 'Чернігівська міська рада', 'Харківська міська рада', 'Івано-Франківська міська рада', 'Луцька міська рада', 'Хмельницька міська рада', 'Полтавська міська рада', 'Вінницька міська рада', 'Ужгородська міська рада', 'Сумська міська рада', 'Миколаївська міська рада', 'Краматорська міська рада', 'Херсонська міська рада', 'Житомирська міська рада', 'Тернопільська міська рада', 'Чернівецька міська рада', 'Рівненська міська рада', 'Кропивницька міська рада', 'Черкаська міська рада', 'Управління житлово-комунального господарства Сєвєродонецької міської ради', 'Управління житлового та комунального господарства Краматорської міської ради', 'КОМПЛЕКСНА ДИТЯЧО-ЮНАЦЬКА СПОРТИВНА ШКОЛА 4 ВІДДІЛУ МОЛОДІ ТА СПОРТУ СЄВЄРОДОНЕЦЬКОЇ МІСЬКОЇ РАДИ']
Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот'

In [502]:
#набір ключових слів-фільтрів для відділення шкільних закупівель

filter_keywords = ['школ', 'шкіль', 'загальноосвітн', 
                   'гімназ', 'ліце[йя]', 'колегіум',
                   'сад[ко][ак]', 'дошкільн', 'яс[ле][ал]']

filter_keywords_strict = ['зош', 'знз', 'сзш', 'нвк', 'увк', 'снвк', 'дюсш', 'днз']

#набір ключових слів-фільтрів для кожної категорії окремо
filter_shkola = ['школ', 'шкіль',  'загальноосвітн',
                 'гімназ',  'ліце[йя]', 'колегіум'] # 'виховн' прибрав

filter_shkola_name = 'Школа'

filter_sadik = ['днз', 'садок', 'дошкільн', ' садка', 'ясла', 'ясел']
filter_sadik_name = 'ДНЗ'

filter_sport = ['юнацька спортивна', 'дюсш', 'дитячо-спортивна']
filter_sport_name = 'ДЮСШ'

filter_music = ['музич', "дмш"]
filter_music_name = 'Музична школа'

filter_art = ['мистецтв', 'дхш']
filter_art_name = 'Школа мистецтв'

filter_hud = ['худож']
filter_hud_name = 'Художня школа'

filter_poza = ['позашкільн', 'міжшкільн', 'МСВЗШ', 'спеціальна вечірня']
filter_poza_name = 'Позашкільне'

filter_inshe = 'Інше'

In [503]:
"""
РЕГУЛЯРНІ ВИРАЗИ

*** lot_regex потребує розширення переліку варіантів і доопрацювання
*** reg_quotes витягує забагато шлаку 
"""

# reg_quotes_update = """((['](.*?)[']{1}?)|([«](.*?)[»]{1}?)|(["](.*?)["]{1}?))"""
reg_number = '(№\s*\d+)' #витягує число перед яким є символ №
lot_regex = '(((\DЗОШ\D)|(\Dзош\D)|(\DСШ\D)|(\DДНЗ\D)|(\DДНЗ\D)).(\d+))' #витягує номер поруч з ключовими словами

In [513]:
""" ФІЛЬТРУВАННЯ ШКІЛЬНИХ ЗАКУПІВЕЛЬ """

columns_to_check = ['Тендер', 'Организатор', 'ОписаниеТендера', 'Лот']
filtered = []

for column in columns_to_check:
    filtered.append(df_raw[df_raw[column].str.contains(r'\b({})'.format('|'.join(filter_keywords)), #пошук за ключовими словами всередині слів
                            case = False,
                            na = False,
                            regex = True)])

for column in columns_to_check:
    filtered.append(df_raw[df_raw[column].str.contains(r'\b({})\b'.format('|'.join(filter_keywords_strict)), #пошук за словами тільки коли це окремі слова
                            case = False,
                            na = False,
                            regex = True)])
    
df_filtered = pd.concat(filtered).drop_duplicates()

print("З-поміж",len(df_raw), "знайдено", len(df_filtered))

C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


З-поміж 592175 знайдено 69271


### Крок 2. Визначення типу школи: Школа, ДНЗ, Спортивна, Музична, Художня, Позашкільне

In [514]:
"""
СТВОРЕННЯ БАЗИ ШКІЛ df_filtered
*** Інтернати не включені

"""


#присвоєння школам-замовникам типу
df_filtered['Тип закладу'] = pd.np.where(
    # 1
    df_filtered["Организатор"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_sport), case = False), 
                filter_sport_name,
    # 2
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_music), case = False), 
                filter_music_name,
    # 3
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_art), case = False),
                filter_art_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_art), case = False),
                filter_art_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_art), case = False), 
                filter_art_name,
    # 4
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_hud), case = False), 
                filter_hud_name,
    # 5
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_sadik), case = False), 
                filter_sadik_name,
    # 6
    pd.np.where(df_filtered["Организатор"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Тендер"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["ОписаниеТендера"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Адрес поставки"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name,
    pd.np.where(df_filtered["Лот"].str.contains('|'.join(filter_shkola), case = False), 
                filter_shkola_name, "Інше" ))))))))))))))))))))))))))))))

print("Всього закупівель:", df_filtered.shape[0])
print("За типом:", df_filtered['Тип закладу'].value_counts())

Всього закупівель: 69271
За типом: ДЮСШ              38804
Школа             16207
ДНЗ               10549
Інше               1732
Музична школа      1048
Школа мистецтв      721
Художня школа       210
Name: Тип закладу, dtype: int64


In [515]:
### Прибирання "позашкільних закладів"

df_filtered = df_filtered[~df_filtered['ОписаниеТендера'].str.contains('|'.join(filter_poza), case = False, na = False)]
df_filtered = df_filtered[~df_filtered['Тендер'].str.contains('|'.join(filter_poza), case = False, na = False)]
df_filtered = df_filtered[~df_filtered['ОписаниеТендера'].str.contains('|'.join(filter_poza), case = False, na = False)]
df_filtered = df_filtered[~df_filtered['Лот'].str.contains('|'.join(filter_poza), case = False, na = False)]
print("Всього закупівель:", df_filtered.shape[0])

Всього закупівель: 69108


### Крок 3. Визначення номеру школи. ТІЛЬКИ НОМЕРУ, БЕЗ НАЗВИ В ЛАПКАХ

In [516]:
# НОМЕР ЗАКЛАДУ
# Можливо, зробити функцію для виконання даної дії

df_filtered['number'] = df_filtered['Организатор'].str.extract(reg_number, expand = False)
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Тендер'].str.extract(reg_number, 
                                                                                 expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['ОписаниеТендера'].str.extract(reg_number, 
                                                                                          expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Лот'].str.extract(reg_number, 
                                                                              expand = False),
                                               df_filtered['number'])
df_filtered['number'] = np.where(df_filtered['number'].isnull(), 
                                               df_filtered['Адрес поставки'].str.extract(reg_number, 
                                                                                         expand = False),
                                               df_filtered['number'])

# Експерименти з регулярними виразами для власних назв шкіл. Якраз тут вилазить багато сміття. Аби це не чистити, його треба максимально пропрацювати

In [527]:
reg_quotes = """["'«](.*?)["'»]{1}?""" #старий варіант. витягує текст в лапках трьох видів
reg_quotes_update = """((['](.*?)[']{1}?)|([«](.*?)[»]{1}?)|(["](.*?)["]{1}?))"""

In [529]:
df_filtered[df_filtered['number'].isnull()]

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,Тип закладу,number
395,UA-2016-11-07-000459-c-L1,UA-2016-11-07-000459-c,"Послуги з організації шкільного харчування (у звязку з відсутністю власного харчоблоку переваги надаються учасникам, які надають послуги в м- р ""Дружба"")",67700,Тернопільський педагогічний ліцей спортивного профілю Тернопільської міської ради Тернопільської області,"Харчування учнів ліцею одноразове (обід), харчування учнів ліцею (трьохразове харчування) (у звязку з відсутністю власного харчоблоку переваги надаються учасникам, які надають послуги в м- р ""Друж...",Послуги з організації шкільного харчування,"Винниченка, 6","Винниченка,6",Тернопільська міська рада,Школа,NaN
1204,UA-2016-11-14-000439-a-L1,UA-2016-11-14-000439-a,Послуги з організації харчування школярів,12363876,Управління освіти адміністрації Індустріального району Харківської міської ради,NaN,Послуги з організації харчування школярів,Загальноосвітні навчальні заклади Індустріального району міста Харкова відповідно до дислокації,"Біблика, 26",Харківська міська рада,ДЮСШ,NaN
1205,UA-2016-11-14-000602-b-L1,UA-2016-11-14-000602-b,Послуги з організації харчування школярів,11507124,Управління освіти адміністрації Слобідського району Харківської міської ради,NaN,Послуги з організації харчування школярів,Загальноосвітні навчальні заклади (далі – ЗНЗ) Управління освіти адміністрації Комінтернівського району Харківської міської ради,"вулиця Плеханівська, 94",Харківська міська рада,ДЮСШ,NaN
1642,UA-2016-12-09-002440-b-L1,UA-2016-12-09-002440-b,Шкільні меблі.,22000,Станція юних техніків - Центр науково-технічної творчості молоді,Відомості про закупівлю в прикріплених документах.,Шкільні меблі,"вул.Кибальчича, 12-а",Кибальчича 12- А,КМДА,Школа,NaN
1651,UA-2016-12-13-000174-a-L1,UA-2016-12-13-000174-a,Шкільні меблі,40000,"Державний професійно-технічний навчальний заклад ""Міжрегіональний центр ювелірного мистецтва м. Києва""","30 Комплектів учнівських 2-місних без полиці, регулюємих по висоті (стіл 1200*500*640-760мм 1 шт., стілець 380х350х380-460мм - 2 шт.) Каркас сірий, з боку знаходиться крючок.\n3 столи учительські ...",Шкільні меблі,вул. Сосніних 13,вул. Сімї Сосніних 13,КМДА,Школа мистецтв,NaN
1657,UA-2016-11-17-000102-c-L1,UA-2016-11-17-000102-c,Шкільні меблі,72000,УПРАВЛІННЯ КУЛЬТУРИ І ТУРИЗМУ ХМЕЛЬНИЦЬКОЇ МІСЬКОЇ РАДИ,NaN,Шкільні меблі,"вул. Степана Бандери, 19","ВУЛИЦЯ ПРОСКУРІВСЬКА, будинок 30",Хмельницька міська рада,ДЮСШ,NaN
1660,UA-2016-10-07-000178-a-L1,UA-2016-10-07-000178-a,Шкільні меблі,83800,Економіко-правовий ліцей ІІ-ІІІ ступенів міста .Києва,Шкільні меблі,Шкільні меблі,"Мілютенка, 5-б",Мілютенка 5-Б,КМДА,Школа,NaN
1662,UA-2016-11-25-000746-c-L1,UA-2016-11-25-000746-c,Шкільні меблі,138000,Станція юних техніків - Центр науково-технічної творчості молоді,Комплект учнівський 2-х місний - 8 компл.;\nКрісло 4-місне м' яке - 36 компл.;\nДошка 2000х1000 - 1шт.,Шкільні меблі,"вул.Кибальчича, 12-а",Кибальчича 12- А,КМДА,Школа,NaN
1664,UA-2016-08-08-000220-b-L1,UA-2016-08-08-000220-b,Шкільні меблі,127000,УПРАВЛІННЯ ОСВІТИ ЧЕРНІВЕЦЬКОЇ МІСЬКОЇ РАДИ,"Стіл учнівський 6 група, стілець учнівський 6 група, стіл учнівський антисколіозний регульований 4-6 група, стілець учнівський 4 група, дошка шкільна 3-х елементна, шафа для роздягальні 8-секційна.",Шкільні меблі,Героїв Майдану 176,Героїв Майдану 176,Чернівецька міська рада,Школа,NaN
1665,UA-2016-11-15-000776-b-L1,UA-2016-11-15-000776-b,Шкільні меблі,28800,Економіко-правовий ліцей ІІ-ІІІ ступенів міста .Києва,Шкільні меблі,Шкільні меблі,"Мілютенка, 5-б",Мілютенка 5-Б,КМДА,Школа,NaN


In [ ]:
# ВЛАСНА НАЗВА
# *** ПІДТЯГУЄ ЗАБАГАТО СМІТТЯ, ТОМУ НАЗВИ В ЛАПКАХ ШУКАЮТЬСЯ ТІЛЬКИ В КОЛОНЦІ "Организатор"
# ***  МОЖНА ВЗЯТИ ПЕРЕЛІК ЗАКЛАДІВ ВІД МОН І ЗРОБИТИ СПИСОК НАЗВ ЗАКЛАДІВ. ДАЛІ ФІЛЬТРУВАТИ ЗА ЦИМ СПИСКОМ ВЖЕ

df_filtered_temp = df_filtered.copy() #створюю резервну базу, де вже алгоритм визначив номери. зручно для тестування

df_filtered_temp['number'] = np.where(df_filtered_temp['number'].isnull(), 
                                               df_filtered_temp['Тендер'].str.extract(reg_quotes_update, 
                                                                                 expand = False),
                                      df_filtered_temp['number'])

df_filtered_temp['number'] = np.where(df_filtered_temp['number'].isnull(), 
                                               df_filtered_temp['Организатор'].str.extract(reg_quotes_update, 
                                                                                         expand = False),
                                               df_filtered['number'])
df_filtered_temp['number'] = np.where(df_filtered_temp['number'].isnull(), 
                                               df_filtered_temp['Лот'].str.extract(reg_quotes_update, 
                                                                                         expand = False),
                                               df_filtered_temp['number'])
df_filtered_temp['number'] = np.where(df_filtered_temp['number'].isnull(), 
                                               df_filtered_temp['ОписаниеТендера'].str.extract(reg_quotes_update, 
                                                                                         expand = False),
                                               df_filtered_temp['number'])

In [ ]:
""" ПОКАЗУЄ, ЩО НАФІЛЬТРУВАЛО """

df_filtered_temp[(~df_filtered_temp.number.str.contains("№", na = False)&df_filtered_temp.number.notnull())]

In [499]:
""" ПОКАЗУЄ, ЗА ЯКИМИ СЛОВАМИ СПРАЦЬОВУЄ РЕГУЛЯРНИЙ ВИРАЗ """

df_filtered_temp['Тендер'].str.extract(reg_quotes_update, expand = False)

,0,1,2,3
4,"""Русанівка""",NaN,NaN,Русанівка
8,"""Русанівка""",NaN,NaN,Русанівка
16,NaN,NaN,NaN,NaN
395,"""Дружба""",NaN,NaN,Дружба
1204,NaN,NaN,NaN,NaN
1205,NaN,NaN,NaN,NaN
1638,NaN,NaN,NaN,NaN
1639,NaN,NaN,NaN,NaN
1640,NaN,NaN,NaN,NaN
1641,NaN,NaN,NaN,NaN


In [419]:
# КІЛЬКІСТЬ ЗАКУПІВЕЛЬ БЕЗ НОМЕРУ ЧИ НАЗВИ
len(df_filtered[df_filtered.number.isnull()])

5750

### Крок 4. Очистка даних

In [517]:
df_filtered.number = df_filtered.number[df_filtered.number. #прибрати пробіли у рядках з №
                                  str.contains('№', 
                                               case = False, 
                                               na = False)].str.replace('\s','')

In [530]:
"""
ПОКАЗАТИ ВСІ НОМЕРИ ТРЬОХЗНАЧНІ
ДЛЯ ПЕРЕВІРКИ reg_number

"""

three_digit_number = "№[1-9][0-9]{0,3}$"

only_numbers = df_filtered.number[df_filtered['number'].
                                  str.contains(three_digit_number, regex = True,
                                               flags=re.IGNORECASE, na = False)].unique().tolist()
only_numbers.sort()
print(len(only_numbers))
only_numbers

724


['№1',
 '№10',
 '№100',
 '№101',
 '№102',
 '№1023',
 '№103',
 '№1030',
 '№104',
 '№105',
 '№1053',
 '№106',
 '№1060',
 '№107',
 '№108',
 '№109',
 '№1094',
 '№11',
 '№110',
 '№111',
 '№1118',
 '№112',
 '№113',
 '№1135',
 '№1138',
 '№114',
 '№115',
 '№116',
 '№117',
 '№118',
 '№119',
 '№12',
 '№120',
 '№121',
 '№122',
 '№123',
 '№124',
 '№125',
 '№126',
 '№127',
 '№128',
 '№1281',
 '№129',
 '№13',
 '№130',
 '№131',
 '№132',
 '№133',
 '№134',
 '№135',
 '№136',
 '№137',
 '№138',
 '№139',
 '№14',
 '№140',
 '№141',
 '№142',
 '№143',
 '№144',
 '№145',
 '№146',
 '№1465',
 '№147',
 '№148',
 '№149',
 '№15',
 '№150',
 '№151',
 '№152',
 '№153',
 '№154',
 '№155',
 '№156',
 '№157',
 '№158',
 '№159',
 '№1591',
 '№16',
 '№160',
 '№1608',
 '№161',
 '№162',
 '№163',
 '№164',
 '№165',
 '№166',
 '№167',
 '№168',
 '№169',
 '№17',
 '№170',
 '№171',
 '№172',
 '№173',
 '№174',
 '№1745',
 '№175',
 '№176',
 '№177',
 '№178',
 '№179',
 '№18',
 '№180',
 '№181',
 '№182',
 '№183',
 '№184',
 '№185',
 '№186',
 '№187',

In [425]:
"""
ПОКАЗАТИ ВСІ НАЗВИ В ДУЖКАХ.
ДЛЯ ПЕРЕВІРКИ reg_quotes

"""

not_numbers = df_filtered.number[~df_filtered['number'].
                                  str.contains('№', case = False, na = False)].unique().tolist()
not_numbers = [x for x in not_numbers if str(x) != 'nan']
not_numbers.sort()
print(len(not_numbers))
not_numbers

1139


['',
 '  Кава,чай  та супутня продукція ',
 '  Макаронні вироби ',
 ' Lenovo, для КПНЗ ',
 ' Інформація на закупівлю',
 ' Інформація на закупівлю (1)',
 ' Інформація про закупівлю',
 ' Інформація про закупівлю.',
 ' Армстронг',
 ' Будинок школярів ',
 ' Водопровідні та санітарно-технічні роботи',
 ' Гужові чи ручні вози, інші транспортні засоби з немеханічним приводом, багажні вози та різні запасні частини',
 ' Електричні побутові прилади ',
 ' Електромонтажні роботи',
 ' Захисник Вітчизни',
 ' Кава ,чай  та супутня продукція ',
 ' Кава,чай та супутня продукція',
 ' Какао,шоколад та цукрові кондитерські вироби ',
 ' Капітальний ремонт пральні навчально – виховного комплексу школи – садок « Арніка',
 ' Київська дитяча  школа  українського танцю ',
 ' Ластівка',
 ' Луцький НВК ЗОШ-інтернат І-ІІІ тсупенів-правознавчий ліцей з посиленою фізпідготовкою ',
 ' Львівське вище професійне училище дизайну та будівництва',
 ' Любисток ',
 ' Макаронні вироби ',
 ' Моноліт',
 ' Монтессорі',
 ' Оболо

In [531]:
"""
Зовнішня база з переліком шкіл 'school_names.csv', згенерована у іншому файлі dovidnyk_minosvity.
Тут зібрані власні назви в лапках, які:
- мають 1 слово
- довші, ніж 4 символи

df_high_attention - показує закупівлі, де назва закладу з бази Міносвіти збігається з власною назвою,
витягнутою регулярними виразами (reg_quote)

"""

df_names = pd.read_csv('school_names.csv', index_col = 0)['0'].tolist() # список закладів від Міносвіти з власними іменами довжиною 1 слово

df_only_names = df_filtered[~df_filtered['number'].str.contains('№', case = False, na = False)&
                      df_filtered['number'].notnull()]

df_high_attention = df_only_names[df_only_names.number.str.contains(r'\b({})\b'.format('|'.join(df_names)),
                            case = False, na = False, regex = True)]

print(len(df_only_names), "залишилось", len(df_high_attention))

df_high_attention.to_excel("School string names.xls")

C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if sys.path[0] == '':


0 залишилось 0


PermissionError: [Errno 13] Permission denied: 'School string names.xls'

### Крок 5. Визначення міста для школи, координат

In [518]:
def cities_organ():
    
    regions = ['Суми', 'Київ', 'Полтава', 'Харків', 'Чернігів', 'Вінниця', 'Львів', 'Миколаїв', 'Дніпро',
            'Чернівці', 'Запоріжжя', 'Одеса', 'Черкаси', 'Херсон', 'Рівне', 'Тернопіль', 'Ужгород', 
            'Луцьк', 'Івано-Франківськ', 'Житомир', 'Кропивницький', 'Хмельницький', 
            'Краматорськ', 'Сєвєродонецьк']
    gol_organ = [
        'Сумська міська рада',
        'КМДА',
        'Полтавська міська рада',
        'Харківська міська рада',
        'Чернігівська міська рада',
        'Вінницька міська рада',
        'Львівська міська рада',
        'Миколаївська міська рада',
        'Дніпровська міська рада',
        'Чернівецька міська рада',
        'Запорізька міська рада',
        'Одеська міська рада',
        'Черкаська міська рада',
        'Херсонська міська рада',
        'Рівненська міська рада',
        'Тернопільська міська рада',
        'Ужгородська міська рада',
        'Луцька міська рада',
        'Івано-Франківська міська рада',
        'Житомирська міська рада',
        'Кропивницька міська рада',
        'Хмельницька міська рада',
        'Краматорська міська рада',
        'Сєвєродонецька міська рада']
    
    city_organ = pd.DataFrame(list(gol_organ), columns = ['gol_organ'])
    city_organ['city'] = pd.Series(regions)
    
    return city_organ

In [519]:
city_organ = cities_organ()

In [520]:
df_filtered.columns

Index(['ИдентификаторЛота', 'Идентификатор', 'Тендер', 'СуммаЛота',
       'Организатор', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'Тип закладу', 'number'],
      dtype='object')

### Крок 6. Зведення

In [521]:
"""
Додавання до головної бази: унікальної назви, посилань на закупівлі
 
"""


# колонки назвою школи та міста
df_full_connector = pd.merge(df_filtered, city_organ, left_on = 'Главный орган', right_on = 'gol_organ')
df_full_connector['city_school'] = df_full_connector['city'] + ' ' + df_full_connector['Тип закладу'] + ' ' + df_full_connector['number']

# колонки з посиланнями на закупівлі DOZORRO
df_full_connector['link'] = ('https://dozorro.org/tender/' + df_full_connector['Идентификатор'])

# Перетворення колонки з сумою договору на числовий формат
df_full_connector["СуммаЛота"] = pd.to_numeric(df_full_connector["СуммаЛота"], errors='coerce')

print("Унікальних закладів:", len(df_full_connector['city_school'].unique().tolist()))

Унікальних закладів: 5634


In [361]:
df_full_connector.sample(5)

,ИдентификаторЛота,Идентификатор,Тендер,СуммаЛота,Организатор,ОписаниеТендера,Лот,Адрес поставки,АдресОрганизатора,Главный орган,Тип закладу,number,gol_organ,city,city_school,link
26077,UA-2019-08-12-001095-a-L1,UA-2019-08-12-001095-a,Поточний ремонт навісу над вхідною групою Харк...,56790.15,Управління освіти адміністрації Московського р...,Пусто,Поточний ремонт навісу над вхідною групою Харк...,"вул. Руслана Плоходька, 5-В","вул. Валентинівська, 27-г",Харківська міська рада,Школа,№64,Харківська міська рада,Харків,Харків Школа №64,https://dozorro.org/tender/UA-2019-08-12-001095-a
11744,UA-2017-12-05-001289-c-L1,UA-2017-12-05-001289-c,Дизельне паливо зимове. Для опалення.,3589.00,Дитячо-юнацька спортивна школа з академічного ...,Дизельне паливо зимове. Для опалення. (Бажана ...,Дизельне паливо зимове. Для опалення.,"Труханів острів, вул. Паркова дорога, 14","Труханів острів, Паркова дорога, 14",КМДА,ДЮСШ,NaN,КМДА,Київ,NaN,https://dozorro.org/tender/UA-2017-12-05-001289-c
19737,UA-2016-09-20-000153-c-L1,UA-2016-09-20-000153-c,Книги канцелярські,14000.00,Київська міська клінічна лікарня №5,"ДК 016-2010: 17.23.1 ""Вироби канцелярські, пап...",Книги канцелярські,"вул. Відпочинку, 11","Відпочинку, 11",КМДА,Школа,№5,КМДА,Київ,Київ Школа №5,https://dozorro.org/tender/UA-2016-09-20-000153-c
74585,UA-2018-05-15-000726-a-L1,UA-2018-05-15-000726-a,Виконання робіт з технічного обслуговування сп...,5000.00,Відділ освіти Сєвєродонецької міської ради,Пусто,Виконання робіт з технічного обслуговування сп...,"вул. Вілєсова, буд. 10","ВУЛИЦЯ ВІЛЄСОВА, будинок 10",Сєвєродонецька міська рада,ДНЗ,№43,Сєвєродонецька міська рада,Сєвєродонецьк,Сєвєродонецьк ДНЗ №43,https://dozorro.org/tender/UA-2018-05-15-000726-a
27855,UA-2017-06-27-000281-a-L1,UA-2017-06-27-000281-a,"Проектна продукція ""Реконструкція існуючих теп...",33000.00,ХМЕЛЬНИЦЬКИЙ ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛАД №20 ...,Пусто,"Проектна продукція ""Реконструкція існуючих теп...","вул. Шевченка , 29","ВУЛИЦЯ ШЕВЧЕНКА, будинок 29",Хмельницька міська рада,ДНЗ,№20,Хмельницька міська рада,Хмельницький,Хмельницький ДНЗ №20,https://dozorro.org/tender/UA-2017-06-27-000281-a


In [522]:
df_zakupki = df_full_connector.groupby('city_school')['link'].apply(lambda x: "%s"%', '.join(x))
df_zakupki = pd.DataFrame(df_zakupki)
df_zakupki.reset_index(inplace = True)
df_zakupki.columns
df_zakupki.shape

(5633, 2)

In [523]:
df_suma = df_full_connector.groupby(["city_school"]).sum()
df_suma.reset_index(inplace=True)
df_suma.columns
df_suma.shape

(5633, 2)

In [524]:
df_final = df_suma.merge(df_zakupki, on = 'city_school')
# df_full_connector = pd.merge(df_filtered, city_organ, left_on = 'Главный орган', right_on = 'gol_organ')

In [525]:
pd.set_option('max_colwidth', 200)

In [526]:
df_final

,city_school,СуммаЛота,link
0,Івано-Франківськ Інше №12,2.999100e+05,https://dozorro.org/tender/UA-2018-11-09-000293-a
1,Івано-Франківськ Інше №16,3.283103e+06,"https://dozorro.org/tender/UA-2016-08-25-001100-c, https://dozorro.org/tender/UA-2017-12-01-000872-c, https://dozorro.org/tender/UA-2017-12-22-002131-a, https://dozorro.org/tender/UA-2018-11-07-00..."
2,Івано-Франківськ Інше №19,1.350000e+06,https://dozorro.org/tender/UA-2017-08-07-001047-c
3,Івано-Франківськ Інше №2,1.200000e+05,https://dozorro.org/tender/UA-2017-11-06-001765-c
4,Івано-Франківськ Інше №24,2.812040e+05,"https://dozorro.org/tender/UA-2016-12-01-000649-c, https://dozorro.org/tender/UA-2017-02-17-000531-c"
5,Івано-Франківськ Інше №25,1.365000e+06,https://dozorro.org/tender/UA-2017-09-21-001133-a
6,Івано-Франківськ Інше №6,1.208890e+08,"https://dozorro.org/tender/UA-2017-12-15-001744-b, https://dozorro.org/tender/UA-2017-12-15-000825-b, https://dozorro.org/tender/UA-2017-03-07-000404-b, https://dozorro.org/tender/UA-2019-03-05-00..."
7,Івано-Франківськ ДНЗ №1,3.053820e+05,"https://dozorro.org/tender/UA-2018-05-12-000051-c, https://dozorro.org/tender/UA-2018-05-12-000028-c, https://dozorro.org/tender/UA-2019-10-10-001933-b, https://dozorro.org/tender/UA-2019-10-10-00..."
8,Івано-Франківськ ДНЗ №10,1.004083e+06,"https://dozorro.org/tender/UA-2017-08-09-001258-b, https://dozorro.org/tender/UA-2017-10-11-001911-a"
9,Івано-Франківськ ДНЗ №11,1.013670e+05,https://dozorro.org/tender/UA-2018-05-12-000040-c


In [372]:
df_final.rename(columns={'city_school':'Заклад'}, inplace=True)
df_final.rename(columns={'СуммаЛота':'Витрати'}, inplace=True)
df_final.rename(columns={'link':'Закупівлі'}, inplace=True)

In [373]:
df_final.to_csv('24_regions.csv', index = True, header=True)
pd.read_csv('24_regions.csv', index_col = 0)

,Заклад,Витрати,Закупівлі
0,Івано-Франківськ ДНЗ №1,891882.00,"https://dozorro.org/tender/UA-2018-07-03-001704-a, https://dozorro.org/tender/UA-2018-05-12-000051-c, https://dozorro.org/tender/UA-2018-05-12-000028-c, https://dozorro.org/tender/UA-2019-10-10-00..."
1,Івано-Франківськ ДНЗ №10,1536579.20,"https://dozorro.org/tender/UA-2017-08-09-001258-b, https://dozorro.org/tender/UA-2017-10-11-001911-a, https://dozorro.org/tender/UA-2018-07-30-000244-b, https://dozorro.org/tender/UA-2019-07-15-00..."
2,Івано-Франківськ ДНЗ №11,1157585.00,"https://dozorro.org/tender/UA-2017-07-13-001641-b, https://dozorro.org/tender/UA-2018-10-04-000779-c, https://dozorro.org/tender/UA-2018-10-05-000623-c, https://dozorro.org/tender/UA-2018-05-14-00..."
3,Івано-Франківськ ДНЗ №12,4770834.30,"https://dozorro.org/tender/UA-2019-11-12-000764-c, https://dozorro.org/tender/UA-2016-12-27-002197-b, https://dozorro.org/tender/UA-2017-01-20-001908-b, https://dozorro.org/tender/UA-2017-01-20-00..."
4,Івано-Франківськ ДНЗ №14,978814.00,"https://dozorro.org/tender/UA-2017-08-01-002254-b, https://dozorro.org/tender/UA-2017-11-17-000633-a, https://dozorro.org/tender/UA-2018-08-02-001076-b, https://dozorro.org/tender/UA-2018-12-10-00..."
5,Івано-Франківськ ДНЗ №15,759643.00,"https://dozorro.org/tender/UA-2016-11-03-000230-c, https://dozorro.org/tender/UA-2017-09-27-000346-a, https://dozorro.org/tender/UA-2018-07-02-001188-a, https://dozorro.org/tender/UA-2018-05-12-00..."
6,Івано-Франківськ ДНЗ №16,2055477.90,"https://dozorro.org/tender/UA-2016-09-26-000265-a, https://dozorro.org/tender/UA-2016-09-23-000350-b, https://dozorro.org/tender/UA-2017-11-23-003497-a, https://dozorro.org/tender/UA-2018-07-24-00..."
7,Івано-Франківськ ДНЗ №17,97400.00,https://dozorro.org/tender/UA-2019-03-29-000986-b
8,Івано-Франківськ ДНЗ №18,7698448.80,"https://dozorro.org/tender/UA-2017-09-28-001124-c, https://dozorro.org/tender/UA-2018-11-14-002365-a, https://dozorro.org/tender/UA-2019-08-29-001152-c, https://dozorro.org/tender/UA-2017-02-13-00..."
9,Івано-Франківськ ДНЗ №19,462239.79,"https://dozorro.org/tender/UA-2017-07-20-001550-b, https://dozorro.org/tender/UA-2017-05-30-001105-b, https://dozorro.org/tender/UA-2018-06-12-000192-a, https://dozorro.org/tender/UA-2018-07-05-00..."
